# Исследование надежности заемщиков.

**Oписание задачи**

Необходимо проверить, влияет ли семейное положение и количество детей на погашение кредита в срок.


### Шаг 1. Обзор данных

In [1]:
import pandas as pd

In [2]:
try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('C:/Users/admin/Desktop/1 часть/Предобработка данных. Исследование надежности заемщиков/data.csv')
#data.tail(60)

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Посмотрев таблицу можно заметить, что:** 
- имеются пропуски в столбцах days_employed и total_income;
- столбец days_employed полностью состоит из отрицательных значений;
- в столбце days_employed есть аномалии и в основном они у пенсионеров;
- в столбце children есть отрицательные значения и выбросы, например 20 детей;
- в столбце purpose имеются неявные дубликаты;
- в столбце total_income вещественный тип данных;
- в столбцах education, family_status и gender есть как нижний регистр, так и верхний.

### Шаг 2.1 Заполнение пропусков


In [4]:
# скорректировал столбец days_employed с помощью модуля
data['days_employed'] = abs(data['days_employed'])
# доля пропусков от общего кол-ва строк
share_pass = 1 - data['days_employed'].count() / data['dob_years'].count()
print(f'Доля пропущенных значений в столбцах days_employed и total_income составляет: {share_pass:.0%}')

Доля пропущенных значений в столбцах days_employed и total_income составляет: 10%


In [5]:
data.isna().mean()

children            0.000000
days_employed       0.100999
dob_years           0.000000
education           0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
total_income        0.100999
purpose             0.000000
dtype: float64

# Причины появления пропусков в данных.
Пропуски в основном у пенсионеров и госслужащих.
Невнимательность при заполнении анкеты в столбце days_employed, либо сокрытие данных в случае госслужащего.
В столбце total_income в случае госслужащего - сокрытие доходов, пенсионер - не работает.

In [6]:
# расчет медиан для столбцов с пропусками
median_days_employed = data['days_employed'].median()
print(f'Медианное значение для столбца days_employed: {median_days_employed}')
median_total_income = data['total_income'].median()
print(f'Медианное значение для столбца total_income: {median_total_income}')

# заполнение пропусков медианным значением
data['days_employed'] = data['days_employed'].fillna(value = median_days_employed)
#data['total_income'] = data['total_income'].fillna(value = median_total_income)
data['total_income'] = data['total_income'].fillna(data.groupby('income_type')['total_income'].transform("median"))
#print(data.groupby('income_type')['total_income'].transform("median"))

Медианное значение для столбца days_employed: 2194.220566878695
Медианное значение для столбца total_income: 145017.93753253992


In [7]:
# проверка на заполненность столбцов
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Почему заполнить пропуски медианным значением — лучшее решение для количественных переменных?**

Среднее значение некорректно характеризует данные, когда имеются сильные отличия значений в выборке данных. 
Сотрудники с большим опытом будут получать в разы больше, чем те, кто начал свой карьерный путь после университета. 
Также для столбца days_employed некорректно брать среднее значение, т.к. там есть аномалии.

### Шаг 2.2 Проверка данных на аномалии и исправления.

In [8]:
data.head(50)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [9]:
# замена аномальных значений у пенсионеров в столбце days_employed на те же значения, сокращенные в 24 раза. 
data.loc[(data['income_type'] == 'пенсионер') & (data['days_employed'] > median_days_employed), 'days_employed'] = data['days_employed'] / 24

# проверил, что аномалии есть и у безработных
#display(data[data['days_employed'] > 50000])
# замена аномальных значений у безработных в столбце days_employed на медианные
data.loc[data['income_type'] == 'безработный', 'days_employed'] = median_days_employed

In [10]:
#data.head(50)

### Шаг 2.3. Изменение типов данных.

In [11]:
# Замена вещественного типа данных на целочисленный в столбце total_income
data['total_income'] = data['total_income'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  int32  
 11  purpose           21525 non-null  object 
dtypes: float64(1), int32(1), int64(5), object(5)
memory usage: 1.9+ MB


### Шаг 2.4. Удаление дубликатов.

In [12]:
#Все названия в столбцaх строчными буквами
data['education'] = data['education'].str.lower()
data['family_status'] = data['family_status'].str.lower()
data['income_type'] = data['income_type'].str.lower()
data['gender'] = data['gender'].str.lower()
data['purpose'] = data['purpose'].str.lower()


# Поиск и удаление явных дубликатов
print(f'Количество явных дубликатов: {data.duplicated().sum()}') 
data = data.drop_duplicates().reset_index(drop=True) # удаление явных дубликатов
print(f'Количество явных дубликатов после их удаления: {data.duplicated().sum()}')

Количество явных дубликатов: 71
Количество явных дубликатов после их удаления: 0


In [13]:
# Поиск неявных дубликатов
data['children'] = abs(data['children'])
data.loc[data['children'] == 20, 'children'] = 2 # в столбце children удалил неявные дубликаты
print(data['children'].value_counts())
print()
print(data['education'].value_counts())
print()
print(data['family_status'].value_counts())
print()
print(data['income_type'].value_counts())
print()
print(data['purpose'].value_counts()) # не стал форматировать этот столбец, т.к. потом с ним какие-то операции будут 
print()
print(data['debt'].value_counts())


0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

среднее                15172
высшее                  5250
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

женат / замужем          12339
гражданский брак          4151
не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

сотрудник          11084
компаньон           5078
пенсионер           3829
госслужащий         1457
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи      

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [14]:
data_education = data[['education_id', 'education']] #создал таблицу со столбцами 'education_id' и 'education' из data
data_education = data_education.drop_duplicates().reset_index(drop=True) #удалил дубликаты, т.к. степеней всего 5, а строк много

data_family_status = data[['family_status_id', 'family_status']] #проделал тоже самое с другой категорией
data_family_status = data_family_status.drop_duplicates().reset_index(drop=True)

data = data.drop(['education', 'family_status'], axis = 1) # удаление столбцов ['education', 'family_status'] *посмотрел в инете
#data

### Шаг 2.6. Категоризация дохода.

In [15]:
def total_income_category(row): #создаю функцию, чтобы она прошлась по каждой строке [total_income] 
    if  row['total_income'] <= 30000: #и вывела в новом столбце категорию по жемесячному доходу
        return 'e'
    if 30001 <= row['total_income'] <= 50000:
        return 'd'
    if 50001 <= row['total_income'] <= 200000:
        return 'c'
    if 200001 <= row['total_income'] <= 1000000:
        return 'b'       
    return 'a'
    
data['total_income_category'] = data.apply(total_income_category, axis=1) #вывод нового столбца с категориями по [total_income] 
print(data['total_income_category'].value_counts())

c    16015
b     5042
d      350
a       25
e       22
Name: total_income_category, dtype: int64


### Шаг 2.7. Категоризация целей кредита.

In [16]:
def purpose_category(row): #создаю ф-цию для создания нового столбца с категориями по операциям
    if 'авто' in  row:
        return 'операции с автомобилем'
    if 'жи' in row:
        return 'операции с недвижимостью'
    if 'свадьб' in row:
        return 'проведение свадьбы'
    return 'получение образования'

data['purpose_category'] = data['purpose'].apply(purpose_category)
print(data['purpose_category'].value_counts()) # Около 50% всех запросов связаны с операциями c недвижимостью

data = data.drop(['purpose'], axis = 1) #удаление столбца, т.к. считаю, что инф-я лишняя   

операции с недвижимостью    10811
операции с автомобилем       4306
получение образования        4013
проведение свадьбы           2324
Name: purpose_category, dtype: int64


In [17]:
#data

##### Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [18]:
children_pivot_debt = data.pivot_table(index=['children'], values=['debt'], aggfunc='sum') # те кто имеет долг в зав-ти от кол-ва детей
#display(children_pivot_debt) 

children_pivot_client = data.pivot_table(index=['children'], values=['debt'], aggfunc='count') #общее кол-во клиентов 
#display(children_pivot_total)

children_pivot_total = children_pivot_debt.merge(children_pivot_client, on='children')
children_pivot_total = children_pivot_total.rename(columns={'debt_x':'clients_with_debt', 'debt_y':'total_clients'})
children_pivot_total['percent_debt'] = children_pivot_total['clients_with_debt'] / children_pivot_total['total_clients']
children_pivot_total.sort_values(by='percent_debt')

,clients_with_debt,total_clients,percent_debt
children,,,
5,0,9,0.000000
0,1063,14091,0.075438
3,27,330,0.081818
1,445,4855,0.091658
2,202,2128,0.094925
4,4,41,0.097561


##### Вывод 1:

Максимальная вероятность того, что будет задолженность у людей имеющих 4 ребенка; минимальная - имеющих 5 детей.

##### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [19]:
data_family_status

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,не женат / не замужем


In [20]:
family_pivot = data.pivot_table(index=['family_status_id'], values=['debt'], aggfunc='mean') 
family_pivot.sort_values(by='debt')

,debt
family_status_id,
2,0.065693
3,0.071130
0,0.075452
1,0.093471
4,0.097509


##### Вывод 2:

Максимальная вероятность того, что будет просрочка у людей не состоявших в браке; минимальная - у вдовцов/вдовец.

##### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

0–30000 — 'E';
30001–50000 — 'D';
50001–200000 — 'C';
200001–1000000 — 'B';
1000001 и выше — 'A'.

In [21]:
total_income_pivot = data.pivot_table(index=['total_income_category'], values=['debt'], aggfunc='mean')
total_income_pivot.sort_values(by='debt')

,debt
total_income_category,
d,0.060000
b,0.070607
a,0.080000
c,0.084920
e,0.090909


##### Вывод 3:

Максимальная вероятность того, что будет просрочка у людей c доходом до 30000; минимальная - от 30000 до 50000.

##### Как разные цели кредита влияют на его возврат в срок?

In [22]:
purpose_pivot = data.pivot_table(index=['purpose_category'], values=['debt'], aggfunc='mean')
purpose_pivot.sort_values(by='debt')

,debt
purpose_category,
операции с недвижимостью,0.072334
проведение свадьбы,0.080034
получение образования,0.092200
операции с автомобилем,0.093590


##### Вывод 4:

Максимальная вероятность того, что будет просрочка у людей c операциями связанные с автомобилем; минимальная - с недвижимостью.

## **Влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок?**

In [23]:
conclusion_pivot = data.pivot_table(index=['family_status_id'], columns=['children'], 
                                    values=['debt'], aggfunc='mean', fill_value=0)
#conclusion_pivot.sort_values(by={'debt','children'}) #если можешь, подскажи, пожалуйста, как отсортировать сводную таблицу
conclusion_pivot

debt                                           
children                 0         1         2         3         4  5
family_status_id                                                     
0                 0.069095  0.082224  0.093552  0.068273  0.103448  0
1                 0.083883  0.118000  0.092958  0.142857  0.000000  0
2                 0.062574  0.086420  0.125000  0.000000  0.000000  0
3                 0.070153  0.066456  0.096386  0.090909  0.000000  0
4                 0.092838  0.114537  0.119048  0.125000  0.500000  0

In [24]:
data_family_status

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,не женат / не замужем


##### Общий вывод:

В заключении скажу, что чем больше детей, тем больше вероятность не погашения кредита в срок. 
Также можно сказать, что женатые/замужние клиенты имеют меньшую вероятность просрочки не в зависимости от кол-ва детей.

Клиент с наибольшей вероятностью просрочки - это клиент, имеющий 4-ых детей, не состоявший в браке, с зарплатой до 30000 и мечтающий о машине :)